In [ ]:
# IMPORTS=
import networkx as nx
import matplotlib.pyplot as plt
import folium

# MY_IMPORTS=
from src import Graph as G

In [ ]:
graph = G.Graph('src/data/input/railway.csv')

In [ ]:
# Create a NetworkX graph
gnx = nx.Graph()
for node in graph.nodes:
    gnx.add_nodes_from([(node.id, {'lat': node.position.lat, 'lon': node.position.lon})])
for edge in graph.edges:
    gnx.add_edges_from([(edge.fromNode.id, edge.destNode.id, {'fromLat': edge.fromNode.position.lat, 'fromLon': edge.fromNode.position.lon, 'destLat': edge.destNode.position.lat, 'destLon': edge.destNode.position.lon})])

In [ ]:
# Using nodes lat and lon as positions, draw the graph
pos = {node[0]: (node[1]['lon'], node[1]['lat']) for node in gnx.nodes(data=True)}
nx.draw_networkx_nodes(gnx, pos, node_size=1, node_color='red')
nx.draw_networkx_edges(gnx, pos, edge_color='black', width=0.2)
# plt.show()

In [ ]:
# Create a NetworkX graph
gnx = nx.Graph()
for node in graph.nodes:
    gnx.add_nodes_from([(node.id, {'lat': node.position.lat, 'lon': node.position.lon})])
for edge in graph.edges:
    gnx.add_edges_from([(edge.fromNode.id, edge.destNode.id, {'fromLat': edge.fromNode.position.lat, 'fromLon': edge.fromNode.position.lon, 'destLat': edge.destNode.position.lat, 'destLon': edge.destNode.position.lon})])
pos = {node[0]: (node[1]['lon'], node[1]['lat']) for node in gnx.nodes(data=True)}
nx.draw_networkx_nodes(gnx, pos, node_size=1, node_color='red')
nx.draw_networkx_edges(gnx, pos, edge_color='black', width=0.2)


# Sort the df by the st_no:
graph.df.sort_values(by=['st_no'], inplace=True, ascending=False)
top20trains = []
for train in graph.df['train'].unique():
    top20trains.append(train)
    if len(top20trains) == 20:
        break
graph.df.sort_values(by=['train', 'st_no'], inplace=True, ascending=True)
current_train = ''
current_color = ''
top20paths = []
edges = []
for index, row in graph.df.iterrows():
    if row.train in top20trains:
        if row.train != current_train:
            top20paths.append(edges)
            edges = []
            current_train = row.train
        if (not (row['stay_time'] == '-' and graph.df.iloc[index - 1]['stay_time'] == '-') and (row['train'] == graph.df.iloc[index - 1]['train'])):
            fromNode = G.Node(graph.df.iloc[index - 1]['st_id'], G.Position(graph.df.iloc[index - 1]['lat'], graph.df.iloc[index - 1]['lon']), [])
            destNode = G.Node(row['st_id'], G.Position(row['lat'], row['lon']), [])
            edge = G.Edge(index, fromNode, destNode, row['mileage'], 0)
            edges.append(edge)
top20paths.pop(0)

networkx_colors = ['yellow']
for i in range(len(top20paths)):
    gnx2 = nx.Graph()
    color = networkx_colors[i%len(networkx_colors)]
    for edge in top20paths[i]:
        gnx2.add_nodes_from([(edge.fromNode.id, {'lat': edge.fromNode.position.lat, 'lon': edge.fromNode.position.lon})])
        gnx2.add_nodes_from([(edge.destNode.id, {'lat': edge.destNode.position.lat, 'lon': edge.destNode.position.lon})])
        gnx2.add_edges_from([(edge.fromNode.id, edge.destNode.id, {'fromLat': edge.fromNode.position.lat, 'fromLon': edge.fromNode.position.lon, 'destLat': edge.destNode.position.lat, 'destLon': edge.destNode.position.lon})])
    pos2 = {node[0]: (node[1]['lon'], node[1]['lat']) for node in gnx2.nodes(data=True)}
    nx.draw_networkx_nodes(gnx2, pos2, node_size=1, node_color=color)
    nx.draw_networkx_edges(gnx2, pos2, edge_color=color, width=0.2)
plt.show()

# FOLIUM

In [ ]:
# Create a map of China
map = folium.Map(tiles='Stamen Terrain', width='100%', height='100%', zoom_start=6, location=[35, 105])
nodes = folium.FeatureGroup(name='Nodes')
edges = folium.FeatureGroup(name='Edges')

# ADD NODES TO FOLIUM:
for node in graph.nodes:
    # If the node has no position, do not add it to the map:
    if node.position is None:
        continue
    # Add a marker for each node:
    popup = folium.Popup('<b>Node ID:</b> ' + str(node.id), max_width=200)
    nodes.add_child(folium.CircleMarker(location=[node.position.lat, node.position.lon], radius=7, popup=popup, fill_color='red', color='black', fill_opacity=0.7))

# ADD EDGES TO FOLIUM:
for edge in graph.edges:
    ptA = [edge.fromNode.position.lat, edge.fromNode.position.lon]
    ptB = [edge.destNode.position.lat, edge.destNode.position.lon]
    # If either of the nodes has no position, do not add the edge to the map:
    if None in ptA or None in ptB:
        continue
    # Add a line for each edge:
    edges.add_child(folium.PolyLine(locations=[ptA, ptB], weight=1, color='black', opacity=0.3))

# Add the feature groups to the map:
map.add_child(edges)
map.add_child(nodes)
# Add a layer control to the map:
folium.LayerControl().add_to(map)
# Save the map to an HTML file:
map.save('src/data/output/map.html')